# AutoGen Pangunahing Halimbawa

Sa halimbawang code na ito, gagamitin mo ang [AutoGen](https://aka.ms/ai-agents/autogen) AI Framework upang lumikha ng isang simpleng ahente.

Ang layunin ng halimbawang ito ay ipakita sa iyo ang mga hakbang na gagamitin natin sa mga susunod na halimbawang code kapag ipinatutupad ang iba't ibang mga pattern ng ahente.


## I-import ang Mga Kailangan na Python Package


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## Gumawa ng Kliyente

Sa halimbawang ito, gagamitin natin ang [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) para sa pag-access sa LLM.

Ang `model` ay tinukoy bilang `gpt-4o-mini`. Subukang palitan ang model sa ibang model na makikita sa GitHub Models marketplace upang makita ang iba't ibang resulta.

Bilang mabilisang pagsubok, magpapatakbo lang tayo ng simpleng prompt - `Ano ang kabisera ng France`.


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Pagpapakilala sa Agent

Ngayon na na-set up na natin ang `client` at nakumpirma na gumagana ito, gumawa tayo ng `AssistantAgent`. Ang bawat agent ay maaaring magkaroon ng:  
**name** - Isang maikling pangalan na magagamit para madaling ma-refer ito sa mga multi-agent na daloy.  
**model_client** - Ang client na ginawa mo sa naunang hakbang.  
**tools** - Mga tool na maaaring gamitin ng Agent upang makumpleto ang isang gawain.  
**system_message** - Ang metaprompt na naglalarawan sa gawain, kilos, at tono ng LLM.  

Maaari mong baguhin ang system message upang makita kung paano tumutugon ang LLM. Tatalakayin natin ang `tools` sa Lesson #4.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## Patakbuhin ang Agent

Ang sumusunod na function ay magpapatakbo ng agent. Ginagamit natin ang `on_message` na method upang i-update ang estado ng Agent gamit ang bagong mensahe.

Sa kasong ito, ina-update natin ang estado gamit ang bagong mensahe mula sa user na `"Plan me a great sunny vacation"`.

Maaari mong baguhin ang nilalaman ng mensahe upang makita kung paano naiiba ang tugon ng LLM.


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama't sinisikap naming maging tumpak, pakitandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi pagkakatugma. Ang orihinal na dokumento sa kanyang katutubong wika ang dapat ituring na opisyal na sanggunian. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na maaaring magmula sa paggamit ng pagsasaling ito.
